In [1296]:
import tweepy
import pandas as pd
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAGvbawEAAAAAxvxnJu9E%2F%2FPCe5Oz0irJC5BDvZM%3DEsqky8zPsYhadIeHKzCVsaB0UXGz28FFb2UBqtnNDOJkmfAkfN")
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime
import re
import flair
import yfinance as yf
import nltk
from textblob import TextBlob as tb
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\peeta\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [212]:
elon_musk=44196397

In [216]:
#def user_tweets(user_id, start_time, end_time, iterations):
#    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
#    tweets=[]
#    date=[]
#    a=0
#    for time in datelist:
#        if a>1:
#            user = client.get_users_tweets(user_id, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
#            if type(user.data) != type(None):
#                for i in user.data: 
#                   tweets.append(str(i))
#                    date.append(i.created_at)
#        start=time
#        a=a+1    
#    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets})
#   tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
#   return tweet_df
    
    

In [1327]:
#user_tweets(elon_musk,'2021-01-01', '2022-01-01', 3)

In [1326]:
#elon = client.get_users_tweets(elon_musk, max_results=100, tweet_fields=["created_at"])
#t=[]
#d=[]
#for i in elon.data:
#    t.append(str(i))
#    d.append(i.created_at)
#elon_df=pd.DataFrame({'time':d, 'tweet':t})
#elon_df['time']=elon_df['time'].round('1min')
#elon_df=elon_df.sort_values(by='time', ascending=True).set_index('time')
#elon_df

In [1325]:
#response = client.get_users_tweets(elon_musk, max_results=100, tweet_fields=["created_at"])
#for tweet in response.data:
#    print(tweet)

In [1762]:
def basic_count(query):
    count_df=pd.DataFrame(columns=['time', 'count'])
    response = client.get_recent_tweets_count(query, granularity="hour")
    for count in response.data:
        data={'time':count['end'], 'count':count['tweet_count']}
        count_df=count_df.append(data, ignore_index=True)
    return count_df

In [146]:
query='tesla OR tsla'

In [147]:
pos_query='#bitcoin (Moon OR rocket OR bullish OR up OR positive OR gain) -bearish -lose -crash -down -negative'

In [148]:
basic_count(pos_query)

:Curve   [date]   (count)

In [149]:
basic_count(query)

:Curve   [date]   (count)

In [60]:
datelist = pd.date_range(start, end=datetime.today(), periods=100).tolist()

In [623]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
def tweets(start_time, end_time, iterations, query):
    datelist = pd.date_range(start_time, end_time, periods=iterations).tolist()
    tweets=[]
    date=[]
    prob=[]
    sentim=[]
    a=0
    for time in datelist:
        if a>1:
            #Fetching Tweets
            response = client.search_recent_tweets(query, max_results=100, start_time=start, end_time=time, tweet_fields=["created_at"])
            response.data
            if type(response.data) != type(None):
                #Data Cleaning
                for i in response.data:
                    tw=i
                    tw=str(i)
                    tw=tw.lower()
                    tw=re.sub(r"http\S+", '',tw)
                    tw=re.sub(r"www.\S+",'',tw)
                    tw=re.sub("[^A-Za-z0-9]"," ",tw)
                    tw=tw.split()
                    tw=' '.join(word for word in tw)
                    #Sentiment analysis
                    if len(tw) > 0:
                        sentence = flair.data.Sentence(tw)
                        sentiment_model.predict(sentence)
                        probability=sentence.labels[0].score
                        sentiment=sentence.labels[0].value
                        tweets.append(tw)
                        date.append(i.created_at)
                        prob.append(probability)
                        sentim.append(sentiment)
        start=time
        a=a+1
        
    tweet_df=pd.DataFrame({'time':date, 'tweet':tweets, 'probability':prob, 'sentiment':sentim})
    tweet_df=tweet_df.sort_values(by='time', ascending=True).set_index('time')
    return tweet_df

2022-04-07 20:19:16,030 loading file C:\Users\peeta\.flair\models\sentiment-en-mix-distillbert_4.pt


In [625]:
tweet=tweets('2022-04-01', '2022-04-07', 1000, 'btc OR bitcoin OR crypto -is:retweet lang:en')

In [626]:
tweet

,tweet,probability,sentiment
time,,,
2022-04-01 00:16:57+00:00,footiepalz it s funny how so many fomo in tota...,0.753226,NEGATIVE
2022-04-01 00:16:57+00:00,recognizing recent efforts by volunteer contri...,0.998419,POSITIVE
2022-04-01 00:16:58+00:00,chiweeniemom1 congrat on your winnings contact...,0.956659,NEGATIVE
2022-04-01 00:16:58+00:00,pintuid gabung youtube live di pintu 2ndannive...,0.984505,POSITIVE
2022-04-01 00:16:58+00:00,visa launches creator program to push nfts and...,0.977080,POSITIVE
...,...,...,...
2022-04-06 23:59:59+00:00,during that time i still kept that old maxflow...,0.983043,POSITIVE
2022-04-06 23:59:59+00:00,reneeziv crypto hawkk done it s all thanks a l...,0.996516,POSITIVE
2022-04-06 23:59:59+00:00,when xrp moves it turns the entire crypto spac...,0.999936,NEGATIVE


In [627]:
bkp=tweet

In [628]:
tweet.to_csv('btclarge')

In [1614]:
test=tweet
tb_polarity = [tb(i).sentiment[0] for i in test['tweet']]
nltk_polarity = [sid.polarity_scores(i)['compound'] for i in test['tweet']]
test['tb_polarity']=tb_polarity
test['nltk_polarity']=nltk_polarity

In [1743]:
flairtest=test.drop(columns=['tb_polarity','nltk_polarity'])
nltktest=test.drop(columns=['tb_polarity', 'probability', 'sentiment'])

In [1744]:
flairtest=flairtest.pivot_table('probability',['time','tweet'], 'sentiment')
flairtest=flairtest.groupby('time').count()
flairtest=flairtest.reset_index()
flairtest['time']=flairtest['time'].round('180min')
flairtest=flairtest.groupby('time').sum()
flairtest=flairtest.drop([flairtest.index[0], flairtest.index[-1]])

In [1745]:
nltktest=nltktest.reset_index()
nltktest['time']=nltktest['time'].round('60min')
nltktest=nltktest.groupby('time').mean()

In [1746]:
nltktest.hvplot(x='time', y='nltk_polarity')

:Curve   [time]   (nltk_polarity)

In [1682]:
flairtest.hvplot(x='time',y='POSITIVE')

:Curve   [time]   (POSITIVE)

In [1683]:
flairtest['pct']=(flairtest['POSITIVE']/(flairtest['POSITIVE']+flairtest['NEGATIVE']))

In [1684]:
flairtest['volume']=flairtest['POSITIVE']+flairtest['NEGATIVE']

In [1685]:
flairtest['net']=flairtest['POSITIVE']-flairtest['NEGATIVE']

In [1686]:
flairtest

sentiment,NEGATIVE,POSITIVE,pct,volume,net
time,,,,,
2022-04-01 03:00:00+00:00,845,1211,0.589008,2056,366
2022-04-01 06:00:00+00:00,841,1227,0.593327,2068,386
2022-04-01 09:00:00+00:00,683,1283,0.652594,1966,600
2022-04-01 12:00:00+00:00,800,1245,0.608802,2045,445
2022-04-01 15:00:00+00:00,831,1224,0.595620,2055,393
2022-04-01 18:00:00+00:00,860,1201,0.582727,2061,341
2022-04-01 21:00:00+00:00,890,1179,0.569841,2069,289
2022-04-02 00:00:00+00:00,803,1148,0.588416,1951,345
2022-04-02 03:00:00+00:00,824,1227,0.598245,2051,403


In [1687]:
flairtest_chg=flairtest.pct_change()

In [1688]:
flairtest.hvplot(x='time', y='net')

:Curve   [time]   (net)

In [1747]:
def normil(df):
    df = (df-df.min())/(df.max()-df.min())
    return df

In [1748]:
flairtest_normil=normil(flairtest)
flairtest_chg_normil=normil(flairtest_chg)

In [1749]:
nltktest_normil=normil(nltktest)

In [1692]:
flairtest

sentiment,NEGATIVE,POSITIVE,pct,volume,net
time,,,,,
2022-04-01 03:00:00+00:00,845,1211,0.589008,2056,366
2022-04-01 06:00:00+00:00,841,1227,0.593327,2068,386
2022-04-01 09:00:00+00:00,683,1283,0.652594,1966,600
2022-04-01 12:00:00+00:00,800,1245,0.608802,2045,445
2022-04-01 15:00:00+00:00,831,1224,0.595620,2055,393
2022-04-01 18:00:00+00:00,860,1201,0.582727,2061,341
2022-04-01 21:00:00+00:00,890,1179,0.569841,2069,289
2022-04-02 00:00:00+00:00,803,1148,0.588416,1951,345
2022-04-02 03:00:00+00:00,824,1227,0.598245,2051,403


In [1693]:
flairtest_normil

sentiment,NEGATIVE,POSITIVE,pct,volume,net
time,,,,,
2022-04-01 03:00:00+00:00,0.706897,0.463303,0.342167,0.833333,0.364611
2022-04-01 06:00:00+00:00,0.689655,0.536697,0.386850,0.928571,0.418231
2022-04-01 09:00:00+00:00,0.008621,0.793578,1.000000,0.119048,0.991957
2022-04-01 12:00:00+00:00,0.512931,0.619266,0.546948,0.746032,0.576408
2022-04-01 15:00:00+00:00,0.646552,0.522936,0.410578,0.825397,0.436997
2022-04-01 18:00:00+00:00,0.771552,0.417431,0.277187,0.873016,0.297587
2022-04-01 21:00:00+00:00,0.900862,0.316514,0.143871,0.936508,0.158177
2022-04-02 00:00:00+00:00,0.525862,0.174312,0.336047,0.000000,0.308311
2022-04-02 03:00:00+00:00,0.616379,0.536697,0.437728,0.793651,0.463807


In [1694]:
btc_stock.hvplot(y='Close')

:Curve   [Datetime]   (Close)

In [1695]:
flairtest_normil.hvplot(y=('NEGATIVE'))*btc_normil.hvplot(y='Close')

:Overlay
   .Curve.I  :Curve   [time]   (NEGATIVE)
   .Curve.II :Curve   [Datetime]   (Close)

In [1696]:
flairtest.hvplot(y=('POSITIVE','NEGATIVE'))

:NdOverlay   [sentiment]
   :Curve   [time]   (value)

In [1750]:
nltktest_normil.hvplot()*btc_normil.hvplot(y='Close')

:Overlay
   .Curve.I  :Curve   [time]   (nltk_polarity)
   .Curve.II :Curve   [Datetime]   (Close)

In [1698]:
flairtest.hvplot(y=('pct'))

:Curve   [time]   (pct)

In [1699]:
flairtest_normil.hvplot(y='POSITIVE')*btc_normil.hvplot(y='Close')

:Overlay
   .Curve.I  :Curve   [time]   (POSITIVE)
   .Curve.II :Curve   [Datetime]   (Close)

In [1718]:
joined.hvplot(y=['pct','Close'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1761]:
joinednltk.hvplot(y=['nltk_polarity','Close'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1760]:
rolling_nltk.hvplot(y=['nltk_polarity','Close'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1723]:
rolling_3.hvplot(y=['POSITIVE','Close'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1732]:
rolling_3.hvplot(y=['shifted_positive','Close'])

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [1751]:
btc = yf.Ticker("BTC-USD")
btc_stock = btc.history(
    start='2022-04-01',
    end='2022-04-08',
    interval='60m'
)

In [1752]:
btc_chg=btc_stock.pct_change()
btc_chg2=btc_chg.pct_change()

In [1753]:
btc_chg_normil=normil(btc_chg)
btc_chg2_normil=normil(btc_chg2)

In [1754]:
btc_normil=normil(btc_stock)

In [1755]:
btc_normil

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-03-31 14:00:00+00:00,0.998337,0.917749,0.901083,0.927386,0.000000,NaN,NaN
2022-03-31 15:00:00+00:00,0.930410,0.847566,0.925101,0.866571,0.165386,NaN,NaN
2022-03-31 16:00:00+00:00,0.867924,0.784522,0.747493,0.700200,0.000000,NaN,NaN
2022-03-31 17:00:00+00:00,0.698126,0.640551,0.734417,0.723690,0.186131,NaN,NaN
2022-03-31 18:00:00+00:00,0.724036,0.660669,0.722580,0.742854,0.063880,NaN,NaN
...,...,...,...,...,...,...,...
2022-04-07 09:00:00+00:00,0.145931,0.039150,0.116346,0.125916,0.000000,NaN,NaN
2022-04-07 10:00:00+00:00,0.133476,0.027621,0.119593,0.125606,0.000000,NaN,NaN
2022-04-07 11:00:00+00:00,0.124477,0.108887,0.125466,0.196076,0.069061,NaN,NaN


In [1706]:
flairtest_normil

sentiment,NEGATIVE,POSITIVE,pct,volume,net
time,,,,,
2022-04-01 03:00:00+00:00,0.706897,0.463303,0.342167,0.833333,0.364611
2022-04-01 06:00:00+00:00,0.689655,0.536697,0.386850,0.928571,0.418231
2022-04-01 09:00:00+00:00,0.008621,0.793578,1.000000,0.119048,0.991957
2022-04-01 12:00:00+00:00,0.512931,0.619266,0.546948,0.746032,0.576408
2022-04-01 15:00:00+00:00,0.646552,0.522936,0.410578,0.825397,0.436997
2022-04-01 18:00:00+00:00,0.771552,0.417431,0.277187,0.873016,0.297587
2022-04-01 21:00:00+00:00,0.900862,0.316514,0.143871,0.936508,0.158177
2022-04-02 00:00:00+00:00,0.525862,0.174312,0.336047,0.000000,0.308311
2022-04-02 03:00:00+00:00,0.616379,0.536697,0.437728,0.793651,0.463807


In [1724]:
flairtest_normil['shifted_positive'] = flairtest_normil['POSITIVE'].shift(periods=-1)
flairtest_normil['shifted_negative'] = flairtest_normil['NEGATIVE'].shift(periods=-1)
flairtest_normil['shifted_pct'] = flairtest_normil['pct'].shift(periods=-1)

In [1725]:
joined=pd.concat([flairtest_normil,btc_normil], axis=1, join='inner')

In [1726]:
joined

,NEGATIVE,POSITIVE,pct,volume,net,shifted_positive,shifted_negative,shifted_pct,Open,High,Low,Close,Volume,Dividends,Stock Splits
2022-04-01 03:00:00+00:00,0.706897,0.463303,0.342167,0.833333,0.364611,0.536697,0.689655,0.386850,0.370455,0.314516,0.395847,0.406451,0.454688,NaN,NaN
2022-04-01 06:00:00+00:00,0.689655,0.536697,0.386850,0.928571,0.418231,0.793578,0.008621,1.000000,0.442344,0.379638,0.459812,0.472638,0.152658,NaN,NaN
2022-04-01 09:00:00+00:00,0.008621,0.793578,1.000000,0.119048,0.991957,0.619266,0.512931,0.546948,0.552714,0.476933,0.560883,0.538358,0.142412,NaN,NaN
2022-04-01 12:00:00+00:00,0.512931,0.619266,0.546948,0.746032,0.576408,0.522936,0.646552,0.410578,0.542868,0.480261,0.505086,0.570582,0.430764,NaN,NaN
2022-04-01 15:00:00+00:00,0.646552,0.522936,0.410578,0.825397,0.436997,0.417431,0.771552,0.277187,0.757168,0.823411,0.807183,0.891052,0.782010,NaN,NaN
2022-04-01 18:00:00+00:00,0.771552,0.417431,0.277187,0.873016,0.297587,0.316514,0.900862,0.143871,0.866387,0.806968,0.884434,0.843016,0.007055,NaN,NaN
2022-04-01 21:00:00+00:00,0.900862,0.316514,0.143871,0.936508,0.158177,0.174312,0.525862,0.336047,0.799028,0.764174,0.851708,0.832001,0.054479,NaN,NaN
2022-04-02 00:00:00+00:00,0.525862,0.174312,0.336047,0.000000,0.308311,0.536697,0.616379,0.437728,0.825956,0.927769,0.845322,1.000000,0.094454,NaN,NaN
2022-04-02 03:00:00+00:00,0.616379,0.536697,0.437728,0.793651,0.463807,0.614679,0.336207,0.673850,0.886558,0.813578,0.918160,0.895950,0.000000,NaN,NaN
2022-04-02 06:00:00+00:00,0.336207,0.614679,0.673850,0.412698,0.683646,0.344037,0.737069,0.266956,0.892126,0.822848,0.937020,0.904889,0.000000,NaN,NaN


In [1727]:
joined=joined.drop(columns=['Open','High','Low','Dividends','Stock Splits'])

In [1728]:
joined

,NEGATIVE,POSITIVE,pct,volume,net,shifted_positive,shifted_negative,shifted_pct,Close,Volume
2022-04-01 03:00:00+00:00,0.706897,0.463303,0.342167,0.833333,0.364611,0.536697,0.689655,0.386850,0.406451,0.454688
2022-04-01 06:00:00+00:00,0.689655,0.536697,0.386850,0.928571,0.418231,0.793578,0.008621,1.000000,0.472638,0.152658
2022-04-01 09:00:00+00:00,0.008621,0.793578,1.000000,0.119048,0.991957,0.619266,0.512931,0.546948,0.538358,0.142412
2022-04-01 12:00:00+00:00,0.512931,0.619266,0.546948,0.746032,0.576408,0.522936,0.646552,0.410578,0.570582,0.430764
2022-04-01 15:00:00+00:00,0.646552,0.522936,0.410578,0.825397,0.436997,0.417431,0.771552,0.277187,0.891052,0.782010
2022-04-01 18:00:00+00:00,0.771552,0.417431,0.277187,0.873016,0.297587,0.316514,0.900862,0.143871,0.843016,0.007055
2022-04-01 21:00:00+00:00,0.900862,0.316514,0.143871,0.936508,0.158177,0.174312,0.525862,0.336047,0.832001,0.054479
2022-04-02 00:00:00+00:00,0.525862,0.174312,0.336047,0.000000,0.308311,0.536697,0.616379,0.437728,1.000000,0.094454
2022-04-02 03:00:00+00:00,0.616379,0.536697,0.437728,0.793651,0.463807,0.614679,0.336207,0.673850,0.895950,0.000000
2022-04-02 06:00:00+00:00,0.336207,0.614679,0.673850,0.412698,0.683646,0.344037,0.737069,0.266956,0.904889,0.000000


In [1729]:
rolling_3=joined.rolling(window=3).mean()

In [1730]:
corrj=joined.corr()

In [1731]:
corrj

,NEGATIVE,POSITIVE,pct,volume,net,shifted_positive,shifted_negative,shifted_pct,Close,Volume
NEGATIVE,1.000000,-0.712419,-0.953138,0.368131,-0.924554,-0.235714,0.264598,-0.270544,-0.132670,0.169381
POSITIVE,-0.712419,1.000000,0.890793,0.390210,0.926074,0.214080,-0.377366,0.332286,0.110270,-0.276618
pct,-0.953138,0.890793,1.000000,-0.070314,0.996213,0.241342,-0.330577,0.314704,0.133200,-0.232346
volume,0.368131,0.390210,-0.070314,1.000000,0.013934,-0.022247,-0.155379,0.088389,-0.027967,-0.144265
net,-0.924554,0.926074,0.996213,0.013934,1.000000,0.243417,-0.348202,0.326733,0.131215,-0.241281
shifted_positive,-0.235714,0.214080,0.241342,-0.022247,0.243417,1.000000,-0.712275,0.890982,0.016338,-0.179709
shifted_negative,0.264598,-0.377366,-0.330577,-0.155379,-0.348202,-0.712275,1.000000,-0.952949,-0.103503,0.333752
shifted_pct,-0.270544,0.332286,0.314704,0.088389,0.326733,0.890982,-0.952949,1.000000,0.072281,-0.289025
Close,-0.132670,0.110270,0.133200,-0.027967,0.131215,0.016338,-0.103503,0.072281,1.000000,-0.385634
Volume,0.169381,-0.276618,-0.232346,-0.144265,-0.241281,-0.179709,0.333752,-0.289025,-0.385634,1.000000


In [1756]:
joinednltk=pd.concat([nltktest_normil,btc_normil], axis=1, join='inner')

In [1757]:
joinednltk=joinednltk.drop(columns=['Open','High','Low','Dividends','Stock Splits'])

In [1758]:
joinednltk.corr()

,nltk_polarity,Close,Volume
nltk_polarity,1.000000,0.360770,-0.211527
Close,0.360770,1.000000,-0.231209
Volume,-0.211527,-0.231209,1.000000


In [1759]:
rolling_nltk=joinednltk.rolling(window=6).mean()